<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных**  
Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.  
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from tabulate import tabulate

In [3]:
data = pd.read_csv('/datasets/insurance.csv')
display(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
data.duplicated().sum()

153

Дубликаты есть, но для нашей задачи они не мешают, оставляем как есть.

## Умножение матриц

**Ответьте на вопрос и обоснуйте решение.**   
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)  
 a. Изменится. Приведите примеры матриц.  
 b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

_____________________

Основные свойства матриц:

$$
1.а. AA^{-1} = E
$$
$$
1.б. AE = EA = A
$$
$$
2. (AB)^T = B^TA^T
$$
$$
3. (AB)^{-1} = B^{-1}A^{-1}
$$
$$
4. (ABC)^{-1} = (C)^{-1} (AB)^{-1}
$$

1.a. Обратная для квадратной матрицы A (англ. inverse matrix) — матрица A с верхним индексом -1, произведение которой на А равно единичной матрице.  
1.б. Если любую матрицу A умножить на единичную (или наоборот), получится эта же матрица A.   
2. Т ранспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке.  
3. Обратная матрица от произведения двух матриц равна произведению матрицы обратной ко второй на матрицу обратную к первой.

Эти свойства матриц работают только для квадртаных матриц (одного размера).

Формула для весов $w$ (формула обучения):

$$
w = (X^{T}X)^{-1}X^{T}y
$$

**Изменится ли качество линейной регрессии после умножения признаком на обратимую матрицу?**

**Ответ:** Не изменится. 

**Обоснование:**   
Чтобы доказать свой ответ, надо взять формулу обучения (весов $w$) и подставить в нее новую матрицу и получить связь новых весов $w'$ и старых весов $w$. И показать, что формула предсказания $a = Xw = X'w' = a'$.   

Пусть существует матрица $X'$, для нее справедливо равентсво:  

$$
X' = X P
$$
где $P$ - это обратимая матрица (квадратная), т.е. при умножении матрицы на нее же обратную (с верхним индексом -1) получим единичную матрицу (первое свойство обратных матриц).  
Тогда для предсказание и вектор весов у нас будут равны:  

$$
a' = X' w'
$$
$$
w' = (X'^{T}X')^{-1}X'^{T}y 
$$
   
Далее в формулу предсказаний $a'$ подставим все, что выше расписали и заменим $X'$ на $XP$:   

$$
a' = XP((XP)^{T}(XP))^{-1}*(XP)^{T}y
$$

Тогда мы можем раскрыть скобки по формуле транспонирования:   

$$
2. (AB)^T = B^TA^T
$$  


$$
a' = XP((XP)^{T}(XP))^{-1}*(XP)^{T}y = XP(P^TX^TXP)^{-1}*P^TX^Ty
$$
   
   
Используем сочетательное свойство и 3 свойство (в моем списке):
$$
4. (ABC)^{-1} = (C)^{-1} (AB)^{-1}
$$

$$
3. (AB)^{-1} = B^{-1}A^{-1}
$$
   
$$
a' = XP(X^TXP)^{-1}(P^T)^{-1}*P^TX^Ty
$$

$PP^{-1}$ заменим на $E$, и при этом $(P^T)^{-1}P^T = PP^{-1} = E$:    

$$
a' = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}*P^TX^Ty = XE(X^TX)^{-1}EX^Ty
$$

Выходит, что предсказание для новой матрицы:  

$$
a' = X(X^TX)^{-1}X^Ty = Xw = a
$$  

Векторы предсказаний совпали. Значит предсказание не меняется, если умножить матрицу признаков на обратимую матрицу.

## Алгоритм преобразования

Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

Нам нужен алгоритм преобразования данных, по которому сложно будет восстановить персональную информацию. Для этого можно использовать домножение матрицы признаков на случайную обратимую матрицу, что и обеспечит защиту персональных данных, при этом не потеряв качество модели.   
   
Алгоритм: (где $Y$ - обратимая матрица, $Z$ - матрица c преобразованными признаками, $X$ - исходные признаки)   
1. Разделяем features и target
2. Генерируем случайную матрицу $Y$.  
2а. Проверяем обратимость этой матрицы $Y$ (для этого надо вычеслить детерминант=определитель).   
3. Получение новой матрицы, путем домножения признаков на случайную обратимую матрицу $Z = YX$  
4. Сравниваем метрику R2 на исходных и зашифрованных признаках

**Обоснование**

Т.к. произведение матриц состоит из всех возможных комбинаций скалярных произведений векторов матрицы features (строк) и векторов матрицы random (столбцов), то получается мы произведем умножение каждого возможного набора признаков (строка-вектор) на наборы векторов random (столбец-вектор), затем сохраним эти произведения в новый вектор-строку. Выходит, что веса вектора-строки итогового будут примерно одинаковые с весами признаков для каждого нового набора векторов в результирующей матрице (матрица обратная для исходной матрицы).

## Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

Разделим данные на обучающую и тестовую выборку и обучим модель.

In [7]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
predictions_origin = model.predict(features_test)
r2_score_origin = r2_score(target_test, predictions_origin)
print("R2 до преобразования =", r2_score_origin)

R2 до преобразования = 0.43522757127026546


Далее создадим функцию для преобразования матрицы признаков.  
**Когда будем вычислять определитель (детерминант), надо чтобы он был не равен нулю - тогда такая матрица будет обратима.**

In [8]:
def transform_features(features):
    new_features = features
    n = features.shape[1] # сохранили в новую переменную features и длинну матрицы (кол-во столбцов [1]) сохранили в n 
    np.random.seed(12345) # задаем начальные условия для генератора случайных чисел
    random_matrix = np.random.randint(1, 10, size = (n,n)) # создали массив(квадратичную матрицу) со случайными числами, размерами как исходный features
    det = np.linalg.det(random_matrix) # вычеслили детерминант матрицы
    
    while det == 0: # если сгенирируется выше рандомная матрица с детерминантом равная 0, то заново надо сгенерировать эту матрицу, так как нам нужны обратимая матрица
        np.random.seed(12345)
        random_matrix = np.random.randint(1, 10, size = (n,n))
        det = np.linalg.det(random_matrix)
        
    new_features = new_features @ random_matrix # перемножили матрицы между собой (рандомную обратимую и матрицу признаков)
    return new_features, random_matrix
    

Применим эту функцию к признакам и посмотрим, что выходит.

In [9]:
print("Признаки до преобразования")
display(features.head())
features, random_matrix = transform_features(features)
print(" ")
print("Признаки после преобразования")
display(features.head())
random_matrix

Признаки до преобразования


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


 
Признаки после преобразования


,0,1,2,3
0,99452.0,396931.0,347287.0,49899.0
1,76279.0,304140.0,266095.0,38329.0
2,42174.0,168087.0,147058.0,21203.0
3,83532.0,333667.0,291948.0,41861.0
4,52371.0,208890.0,182758.0,26301.0


array([[3, 6, 2, 5],
       [6, 3, 2, 7],
       [2, 8, 7, 1],
       [3, 2, 3, 7]])

Видим, что после преобразования признаки стали совсем другими - совсем непонятные числа.   
Теперь вычеслим R2 с новыми преобразованными признаками. 

In [10]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
predictions_transform = model.predict(features_test)
r2_score_transform = r2_score(target_test, predictions_transform)
print("R2 после преобразования =", r2_score_transform)

R2 после преобразования = 0.43522757127030764


In [11]:
col = [["Линейная регрессия до преобразования признаков", r2_score_origin],
       ["Линейная регрессия с преобразованными признаками", r2_score_transform]]

col_names = ['модель', 'R2']
print(tabulate(col, headers=col_names))

модель                                                  R2
------------------------------------------------  --------
Линейная регрессия до преобразования признаков    0.435228
Линейная регрессия с преобразованными признаками  0.435228


R2 метрика не поменялась, значит с поставленной задачей справились. 

**Вывод**  
Разработали метод преобразования данных (перемножили матрицу признаков на случайную обратимую матрицу), чтобы по ним было сложно восстановить персональную информацию. Защитили данные так, что при этом качество модели машинного обучения не ухудшилось.